## LLMs

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
azure_endpoint = os.environ.get('azure_endpoint')

openai_config_list = [
  {
    "model": "gpt-4o",
    "api_key": OPENAI_API_KEY,
    "base_url": azure_endpoint,
  },
  {
    "model": "gpt-4-turbo",
    "api_key": OPENAI_API_KEY,
    "base_url": azure_endpoint,
  }
]

llm_config_openai = {
        "config_list": openai_config_list, 
    }

llm_config = llm_config_openai

In [6]:
with open("bgp_best.txt", "r") as f:
        bgp_best = f.read()

## Autogen

In [ ]:
from autogen import ConversableAgent
from autogen import GroupChat
from autogen import GroupChatManager
from autogen import UserProxyAgent
from autogen import AssistantAgent
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import register_function

Planner = ConversableAgent(
    name="Planner",
    system_message="""You are a software architect. You generate clear and detail coding instructions per requirements for Coder.
     You can generate low level logic to help Coder but you do not write the code""",
    llm_config=llm_config,
)

Coder = AssistantAgent(
    name="Coder",
    system_message="You are a Python developer. You generate code per instructions and fix errors in the code",
    llm_config=llm_config,
)

Critic = AssistantAgent(
    name="Critic",
    system_message="You are a Python code reviewer and critic. You provide recommendations to improve the code",
    llm_config=llm_config,
)

Reporter = AssistantAgent(
    name="Reporter",
    system_message="""You are a Python code reporter and documentor. 
    You write and publish documents about the code so that code is easier to maintain.
    Your report should also include unit results and system test results""",
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    name = "user_proxy",
    human_input_mode = "ALWAYS",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "") and x.get(
        "content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="tmp_dir"),
    },
)



group_chat = GroupChat(
    agents = [user_proxy, Planner, Coder, Critic, Reporter],
    messages = [],
    max_round = 6,
    send_introductions= True,
    speaker_selection_method = "auto",
)

group_chat_manager = GroupChatManager(
    groupchat = group_chat,
    llm_config = llm_config,
)

chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message=f"""You are a team to collaborate on a networking script development project using Python. 
    
    The Python script should perform the following 3 tasks: 

    Task 1: collect device running configuration for BGP, and loopback interfaces. 
    Devices are Cisco IOS-XR routers (provided by routers.csv file; 
    which is a device file, and the order of data is described by the first row of the file,
    (hostname,ip_address,username,password,device_type). You connect to each device by its IP address using the device type provided.
    BGP configuration is collected via 'show running router bgp'.
    Loopback interfaces are collected via 'show ip interface brief | include Loopback'.
    
    Task 2: Compare the BGP running configuration of each device to the best practices provided below and summarize key differences:
    {bgp_best}   
    Ensure all best practices are checked. Save the comparison results.

    Task 3: Make recommendations on what needs to be changed on each device configuration to be compliant with the best practice configuration 
    in a final report. Save the report.
  
    Combine all code blocks into a single script. Add code documentation as needed.
    The project completes when the script executes successfully and the final report is generated.
    Save the final script as main.py.


    """,
    summary_method="reflection_with_llm",
    max_turns=5,
    clear_history=True,
)